In [ ]:
import pandas as pd
from pathlib import Path
import re

### Scenario delta
This scenario calculates the differences between the policy and BAU scenarios.

In [ ]:
# Inputs and outputs
results_path = Path("../results")
tables_path = Path("tables")

In [ ]:
# Load case information
cases = [
    "baseline_2019", "today_2024", "future_2030",
    "policy_co2_100EUR", "policy_co2_200EUR", "policy_co2_500EUR", "policy_co2_750EUR","policy_co2_1000EUR", "policy_co2_1500EUR", "policy_co2_2000EUR",
    "policy_thermic_20pct", "policy_thermic_50pct", "policy_thermic_75pct", "policy_thermic_100pct",
    "policy_elez", "policy_electric"
]

df_cases = []

for case in cases:
    df_partial = pd.read_parquet(results_path / "scenario_solutions/{}.vehicles.parquet".format(case))
    f_electric = df_partial["vehicle_type"].str.endswith("electric")

    df_cases.append({
        "case": case,

        "vehicles": len(df_partial[["vehicle_id", "depot", "operator"]].drop_duplicates()),
        "thermic_vehicles": len(df_partial[~f_electric][["vehicle_id", "depot", "operator"]].drop_duplicates()),
        "electric_vehicles": len(df_partial[f_electric][["vehicle_id", "depot", "operator"]].drop_duplicates()),
        
        "distance_km": df_partial["distance_m"].sum() * 1e-3,
        "thermic_distance_km": df_partial[~f_electric]["distance_m"].sum() * 1e-3,
        "electric_distance_km": df_partial[f_electric]["distance_m"].sum() * 1e-3,

        "fuel_L": df_partial["fuel_L"].sum(),
        "electricity_kWh": df_partial["electricity_Wh"].sum() * 1e-3,
        "energy_kWh": df_partial["energy_Wh"].sum() * 1e-3,
        "co2eq_kg": df_partial["co2eq_g"].sum() * 1e-3,

        "total_cost_EUR": df_partial["total_cost_EUR"].sum(),

        "deliveries": df_partial["deliveries"].sum()
    })

df_cases = pd.DataFrame.from_records(df_cases)
df_cases["cost_per_delivery_EUR"] = df_cases["total_cost_EUR"] / df_cases["deliveries"]

df_cases = df_cases.drop(columns = [
    "deliveries"
])

In [ ]:
# Calculate offset with reference cases
reference_cases = ["baseline_2019", "today_2024"]
ignore_cases = []
df_comparison = []

comparison_columns = ["distance_km", "energy_kWh", "co2eq_kg", "cost_per_delivery_EUR"]

df_comparison = df_cases[~df_cases["case"].isin(reference_cases) & ~df_cases["case"].isin(ignore_cases)]
df_comparison = df_comparison[["case"] + comparison_columns].copy()

df_plot = []

for reference_case in reference_cases:
    year = int(reference_case.split("_")[-1])

    df_partial = df_cases[~df_cases["case"].isin(reference_cases) & ~df_cases["case"].isin(ignore_cases)]
    df_partial = df_partial[["case"] + comparison_columns].copy()

    for column in comparison_columns:
        reference_value = df_cases.loc[df_cases["case"] == reference_case, column].values[0]
        df_comparison["{}_{}".format(column, year)] = 1e2 * (df_comparison[column] - reference_value) / reference_value
        df_partial[column] = 1e2 * (df_partial[column] - reference_value) / reference_value

    df_partial["reference"] = year
    df_plot.append(df_partial)

df_plot = pd.concat(df_plot)

years = [2024, 2019]

df_comparison = df_comparison[["case"] + [
    "{}_{}".format(column, year)
    for year in years for column in comparison_columns
]]

df_comparison.columns = pd.MultiIndex.from_tuples([("case", "")] + [
    [year, column.split("_")[0]]
    for year in years for column in comparison_columns
])

df_comparison.round(2)

In [ ]:
# Prepare table with deltas
df_table = df_comparison.copy()

def replace_variable(variable):
    if variable == "distance": return "Dist."
    if variable == "co2eq": return "Emissions"
    if variable == "energy": return "Energy"
    if variable == "cost": return "Cost"
    return variable

df_table.columns = pd.MultiIndex.from_tuples([("case", "")] + [
    ["$\\Delta {}$".format(year), replace_variable(column.split("_")[0])]
    for year in years for column in comparison_columns
])

df_table = df_table.set_index("case")
df_table.index.name = "[\\%]"

def replace_name(name):
    if name == "baseline_2019": return "Baseline 2019"
    if name == "today_2024": return "Today 2024"
    if name == "future_2030": return "Future 2030"
    if name == "policy_elez": return "Electric LEZ"
    if name == "policy_electric": return "100\\% Electric"

    if name.startswith("policy_co2"):
        return "Carbon tax ({} EUR)".format(name.split("_")[-1].replace("EUR", ""))

    if name.startswith("policy_thermic"):
        return "ICV tax ({}\\%)".format(name.split("_")[-1].replace("pct", ""))

    return name

df_table.index = [replace_name(name) for name in df_table.index]

with open(tables_path / "delta.tex", "w+") as f:
    table = df_table.to_latex(
        caption = "Changes in key indicators compared to reference scenarios [\\%]", 
        label = "tab:delta", sparsify = True,
        float_format = lambda f: "{:.2f}".format(f)
    )

    table = table.replace("Carbon tax (100 EUR)", "\\midrule\nCarbon tax (100 EUR)")
    table = table.replace("ICV tax (20\\%)", "\\midrule\nICV tax (20\\%)")
    table = table.replace("Electric LEZ", "\\midrule\nElectric LEZ")

    table = table.replace("\\begin{table}", "\\begin{table}\n\\centering")
    
    table = table.replace("lrrrrrrrr", "lrrrr|rrrr")
    table = re.sub(r"(-[0-9.]+)", "\\\\textbf{\\1}", table)
    table = table.replace("{r}", "{c}")
    table = table.replace("{$\\Delta 2019$} \\\\", "{$\\Delta 2019$} \\\\ \n \\midrule")

    f.write(table)

df_table